In [3]:
import SimpleITK as sitk
import numpy as np
import os
from radiomics import featureextractor, getTestCase

In [4]:
image_viewer = sitk.ImageViewer()

In [5]:
#slicer = 'D:/Users/maryskal/Slicer 5.0.2/Slicer.exe'
slicer = '/home/maryskal/Slicer/Slicer'
image_viewer.SetApplication(slicer)

# EXTRACTOR
Creamos un extractor default, luego vamos a ver como se pueden ajustar parámetros

In [6]:
extractor = featureextractor.RadiomicsFeatureExtractor()

## IMAGE TYPES

In [7]:
extractor.enabledImagetypes

{'Original': {}}

Es una lista de tipos de imagenes que se van a analizar. Siempre se analiza la original, pero a esta se le pueden aplicar filtros y se vuelven a calcular todas las características tambien sobre esos filtros

### Enable y disable all

In [8]:
extractor.disableAllImageTypes()
extractor.enabledImagetypes

{}

In [9]:
extractor.enableAllImageTypes()
extractor.enabledImagetypes

{'Exponential': {},
 'Gradient': {},
 'LBP2D': {},
 'LBP3D': {},
 'LoG': {},
 'Logarithm': {},
 'Original': {},
 'Square': {},
 'SquareRoot': {},
 'Wavelet': {}}

Las que mas se utilizan son **Original**, **Wavelt** y **LoG**

In [10]:
### Enable y disable por tipos

In [11]:
extractor.disableAllImageTypes()
extractor.enableImageTypeByName('Original')
extractor.enabledImagetypes

{'Original': {}}

In [12]:
extractor.enableImageTypeByName('Wavelet')
extractor.enableImageTypeByName('LoG')
extractor.enabledImagetypes

{'Original': {}, 'Wavelet': {}, 'LoG': {}}

## SETTINGS

In [13]:
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True}

Esto determina como se van a extraer las características. 

Existen los siguientes settings:

- **A. Feature extractor level**
    - ***Image normalization***
        - normalize
        - normalizeScale
        - removeOutliers
    - ***Resampling image/mask***
        - resampledPixelSpacing
        - interpolator
        - padDistance
    - ***Pre-Cropping***
        - preCrop
    - ***Resegmentation***
        - resegmentRange
        - resegmentMode
        - resegmentShape
    - ***Mask validation***
        - minimumROIDimensions
        - minimumROISize
        - geometryTolerance
        - correctMask
    - ***Miscellaneous***
        - additionalInfo
- **B. Filter level**
    - ***Laplacian of Gaussian settings***
        - sigma
    - ***Wavelet settings***
        - start_level
        - level
        - wavelet
    - ***Gradient settings***
        - gradientUseSpacing
    - ***Local Binary Pattern 2D***
        - lbp2DRadius
        - lbp2DSamples
        - lbp2DMethod
    - ***Local Binary Pattern 3D***
        - lbp3DLevels
        - lbp3DIcosphereRadius
        - lbp3DIcosphereSubdivision

- **C. Feature Class Level**
    - Label
    - ***Image discretization***
        - binWidth
        - binCount
    - ***Forced 2D extraction***
        - force2D
        - force2Ddimension
    - ***Texture matrix weighting***
        - weightingNorm
    - ***Distance to neighbour***
        - distances
- **D. Feature Class Specific Settings**
    - ***First Order***
        - voxelArrayShift
    - ***GLCM***
        - symmetricalGLCM
    - ***GLDM***
        - gldm_a
- **E. Voxel-based specific settings**
    - kernelRadius
    - maskedKernel
    - initValue
    - voxelBatch

Los settings los podemos definir en un diccionario y luego inicializar el extractor

In [14]:
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1, 2, 3]

In [15]:
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 20,
 'sigma': [1, 2, 3]}

Tambien se pueden introducir de esta manera

In [16]:
extractor = featureextractor.RadiomicsFeatureExtractor(binWidth=20, sigma=[1, 2, 3])
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 20,
 'sigma': [1, 2, 3]}

## FEATURES
Son las características que vamos a extraer

In [17]:
extractor.enabledFeatures

{'firstorder': [],
 'glcm': [],
 'gldm': [],
 'glrlm': [],
 'glszm': [],
 'ngtdm': [],
 'shape': []}

- **A. First order**
    - energy
    - total energy
    - entropy
    - minimum
    - 10th percentile
    - 90th percentile
    - maximum
    - mean
    - median
    - IQR
    - range
    - mean absolute deviation
    - robust mean absolute deviation
    - roob mean square
    - standar deviation
    - Skewness
    - kurtosis
    - variance
    - uniformity
- **B. Shape**
    - Shape 3D
        - mesh volume
        - voxel volume
        - surface area
        - surface area / volume ratio
        - sphericity
        - compactness 1
        - compactness 2
        - spherical disproportion
        - maximum 3D diameter
        - maximum 2D diameter (slice)
        - maximum 2D diameter (column)
        - maximum 2D diameter (row)
        - major axis length
        - minor axis length
        - least axis length
        - elongation
        - flatness
    - Shape 2D
        - mesh surface
        - pixel surface
        - perimeter
        - perimeter surface ratio
        - sphericity
        - spherical disproportion
        - maximum 2D diameter
        - major axis length
        - minor axis length
        - elongation
- **C. Gray level Co-ocurrence (GLCM)**
    - Autocorrelation
    - Joint Average
    - Cluster Shade
    - Cluster Tendency
    - Contrast
    - Correlation
    - Difference average
    - Difference entropy
    - Difference variance
    - Join energy
    - Join entropy
    - Information measure of corrrelation (IMC 1)
    - IMC 2
    - Inverse difference moment (IDM)
    - Maximal correlation coefficient (MCC)
    - Inverse difference moment normalized (IDMN)
    - Inverse difference (ID)
    - Inverse difference normalized (IDN)
    - Inverse variance
    - Maximum probability
    - Sum average
    - Sum entropy
    - Sum of squares
- **D. Gray level size zone matrix (GLSZM)**
    - Small area emphasis (SAE)
    - Large area emphasis (LAE)
    - Gray level non-uniformity (GLN)
    - Gray level non-uniformity normalized (GLNN)
    - Size zone non uniformity (SZN)
    - Size zone non-uniformity normalized (SZNN)
    - Zone percentage (ZP)
    - Gray level variance (GLV)
    - Zone variance (ZV)
    - Zone entropy (ZE)
    - Low gray level zone emphasis (LGLZE)
    - High gray level zone emphasis (HGLZE)
    - Small area high gray level emphasis (SAHGLE)
    - Large area high gray level emphasis (LAHGLE)
- **E. Gray level run length matrix (GLRLM)**
    - Short run emphasis (SRE)
    - Long run emphasis (LRE)
    - Gray level non-uniformity (GLN)
    - Gray level non-uniformity normalized (GLNN)
    - Run lengh non-uniformity (RLN)
    - Run lengh non-uniformity normalized (RLNN)
    - Run percentage (RP)
    - Gray level variance (GLV)
    - Run variance (RV)
    - Run entropy (RE)
    - Low gray level run enphasis (LGLRE)
    - High gray level run emphasis (HGLRE)
    - Short run low gray level emphasis (SRLGLE)
    - Short run high gray level emphasis (SRLGLE)
    - Long run low gray level emphasis (LRLGLE)
    - Long run high gray level emphasis (LRHGLE)
- **F. Neighbouring gray tone difference matrix (NGTDM)**
    - Coases
    - Contrast
    - Busyness
    - Complexity
    - Strength
- **G. Gray level dependence matrix (GLMD)**
    - Small dependence emphasis (SDE)
    - Large dependence emphasis (LDE)
    - Gray level non-uniformity (GLN)
    - Dependence non-uniformity (DN)
    - Dependence non-uniformity normalized (DNN)
    - Dependence variance (DV)
    - Dependence entropy (DE)
    - Low gray level emphasis (LGLE)
    - High gray level emphasis (HGLE)
    - Small dependences low gray level emphasis (SDLGLE)
    - Small dependences high gray leve emphasis (SDHGLE)
    - Large dependences low gray level emphasis (LDLGLE)
    - Large dependences high gray level emphasis (LDHGLE)

## TODAS

Todas estas cosas se pueden añadir como un diccionario

In [18]:
datos = {'setting': {
            'binWidth': 25,
            'label': 1,
            'interpolator': 'sitkBSpline',
            'resampledPixelSpacing': None,
            'weightingNorm': None},
        'imageType': {
            'Original': {}},
        'featureClass': {
            'shape': None,
            'firstorder': [],
            'glcm': None,
            'glrlm': None,
            'glszm': None,
            'gldm': None}}

In [19]:
extractor = featureextractor.RadiomicsFeatureExtractor(datos)

# EXTRACCIÓN

In [21]:
ctFolder = 'Vessel_stencils'
ctPath = os.path.join(ctFolder,'10047G_INSP_STD_BWH_COPD.nrrd')
maskPath = os.path.join(ctFolder, '10047G_INSP_STD_BWH_COPD','mask__lt5_artery_2.nrrd')

In [22]:
ct = sitk.ReadImage(ctPath)
mask = sitk.ReadImage(maskPath)

In [ ]:
result = extractor.execute(ct, mask)